In [1]:
stats_dict = {
    "DP05_0001E":"total_population",
    "DP05_0032E":"white_population",
    "DP05_0066E":"hispanic_population",
    "DP02_0092E":"foreign_born_population",
    "DP05_0017E":"median_age",
    "DP03_0062E":"median_household_income",
    "DP03_0063E":"mean_household_income",
    "DP03_0128PE":"poverty_population",
    "DP04_0001E":"total_housing_units",
    "DP04_0126E":"units_paying_rent",
    "DP03_0005E":"unemployed_population",
    "DP03_0032E":"working",
    "DP03_0033E":"agriculture_industry",
    "DP03_0039E":"information_industry",
    "DP03_0035E":"manufacturing_industry",
    "DP03_0040E":"financial_industry",
    "DP03_0043E":"entertainment_industry",
    "DP03_0048E":"government_employee",
    "DP02_0063E":"degree_associate",
    "DP02_0064E":"degree_bachelor",
    "DP02_0065E":"degree_higher",
    "DP02_0068E":"veteran_population",
    "DP03_0006E":"armed_forces_population",
    "DP03_0099E":"no_insurance",
    "DP03_0098E":"public_insurance",
    "DP03_0095E":"non_institutionalized_population"
    }
arc_name = "data/stream/2015/04/07"
save_name = "data/twitter_dump.txt"

import geonamescache
gc = geonamescache.GeonamesCache()
internat_cities = [c for c in gc.get_cities().values() if c['population'] >= 100000 and c['countrycode'] != 'US']
for c in internat_cities:
    c['state'] = 'INTERNATIONAL'
import pycountry
countries = [c.name.title() for c in pycountry.countries]
planets = ['Mercury', 'Venus', 'Earth', 'Mars', 'Moon', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto'] #Pluto is not a planet
import requests
from config import census_key

In [2]:
cit_file = "data/uscities.csv"
cities = []
with open(cit_file, "r") as f:
    k = 0
    for line in f:
        if k == 0:
            k=1
            continue
        
        row = [x.replace('\"', '') for x in line.split(',')]
        try:
            temp = {
                'name':row[0],
                'state':row[2],
                'longitude':float(row[8]),
                'latitude':float(row[9]),
                'population':int(round(float(row[10]))),
                'density':float(row[11])
            }
            cities.append(temp)
        except:
            temp = {
                'name':', '.join(row[0:2]),
                'state':row[4],
                'longitude':row[10],
                'latitude':row[11],
                'population':int(round(float(row[12]))),
                'density':float(row[13])
            }
            cities.append(temp)
city_names = {}
for c in cities:
    if c['name'] in city_names.keys():
        city_names[c['name']][c['state']] = c
    else:
        city_names[c['name']] = {c['state']:c}
for c in internat_cities:
    if c['name'] in city_names.keys():
        city_names[c['name']]['state'] = c

In [3]:
abbr_to_states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state_to_abbr = {v.upper():k for k,v in abbr_to_states.items()}

In [10]:
wash_dc = city_names['Washington']['DC']
nyc = city_names['New York']['NY']

def match_country(st):
    if st.lower() in ['us', 'usa', 'america', 'united states', 'united states of america', 'amerika', 'murica', "'murica"]:
        return 'USA'
    else:
        return None

def match_state(st):
    stat = st.upper()
    if 'USA' in stat:
        stat = stat.replace('USA', '').strip()
    if 'AMERICA' in stat:
        stat = stat.replace('AMERICA', '').strip()
    if 'US' in stat:
        stat = stat.replace('US', '').strip()
    if stat in state_to_abbr.keys():
        return state_to_abbr[stat]
    elif stat not in state_to_abbr.values():
        return None
    return stat

def match_city(cit, st = None):
    city = cit.title()
    if city in ['Nederland', 'Russia', 'Scotland', 'Wales', 'England', 'Nowhere', 'Uncertain']:
        return None
    if city == 'New York City' or cit == 'Nyc':
        return nyc
    if st is None:
        if city in countries or city in planets:
            return None
        out = city_names.get(city, None)
        if out and len(out.keys()) > 1:
            temp = 0
            for c in out.values():
                if c['population']>temp:
                    temp = c['population']
                    oout = c
            if oout['state'] != 'INTERNATIONAL' and oout['population'] >= 10000:
                return oout
            else:
                return None
        elif out and len(out.keys())==1:
            return list(out.values())[0]
        else:
            return None
    else:
        #print(cit, st)
        if st == 'DC':
            return wash_dc
        out = city_names.get(cit, None)
        if out:
            out = out.get(st, None)
            if out:
                return out
            else:
                return None
        else:
            return None

def match_string(st):
    st = st.strip()
    st = st.replace('.', '')
    if st.lower() == 'la':
        return match_string('Los Angeles, CA')
    if st.lower() == 'sf':
        return match_string('San Francisco, CA')
    if st.lower() == 'nyc':
        return nyc
    if 'nova iorque' in st.lower():
        return nyc
    if 'washington' in st.lower() and 'dc' in st.lower().replace('.',''):
        return wash_dc
    if st.count(',') == 0:
        try:
            out = match_country(st)
            if out:
                return out
        except:
            pass
        try:
            out = match_state(st)
            if out:
                return out
        except:
            pass
        try:
            out = match_city(st)
            if out:
                return out
        except:
            pass
        if st.count(' ') > 0:
            temp = st.split(' ')
            if len(temp) == 2:
                return match_string(', '.join(temp))
            else:
                return match_string(' '.join(temp[0:-1]) + ', ' + temp[-1])
        return None
    elif st.count(',') == 1:
        cit, st = st.split(',')
        cit = cit.strip().title()
        st = st.strip().upper()
        if match_country(st):
            if match_state(cit.upper()):
                return match_state(cit.upper())
            elif match_city(cit):
                return match_city(cit)
            else:
                return 'USA'
        elif match_state(st):
            st = match_state(st)
            if match_city(cit,st):
                return match_city(cit,st)
            else:
                return None

        else:
            return None
    elif st.count(',') == 2:
        cit, st, ct = st.split(',')
        if match_country(ct.strip()):
            return match_string(','.join([cit,st]))
        else:
            return None

In [5]:
def parse_census_city(city_name):
    city, state = city_name.split(', ')
    city = ' '.join(city.split(' ')[:-1])
    state = state_to_abbr.get(state.upper(), None)
    return city, state

stats_name_map = {v:k for k,v in stats_dict.items()}
url = "https://api.census.gov/data/2017/acs/acs5/profile?get="
url += ",".join(stats_dict.keys())
url += ",NAME&for=place:*&in=state:*"
url += '&key=' + census_key
response = requests.get(url)
print(response.status_code)
if response.status_code < 400:
    data = response.json()
    my_data = data
    ind = data[0].index("NAME")
    stats_ind = {s:data[0].index(s) for s in stats_dict.keys()}
    for data_point in data[1:]:
        try:
            city, state = parse_census_city(data_point[ind])
        except:
            print(data_point[ind])
            continue
        if state and match_city(city, state):
            for k,v in stats_ind.items():
                city_names[city][state][stats_dict[k]] = data_point[v]

url = url = "https://api.census.gov/data/2017/acs/acs5/profile?get="
url += ",".join(stats_dict.keys())
url += ",NAME&for=state:*"
url += '&key=' + census_key
state_stats = {}
response = requests.get(url)
if response.status_code < 400:
    data = response.json()
    ind = data[0].index("NAME")
    stats_ind = {s:data[0].index(s) for s in stats_dict.keys()}
    for data_point in data[1:]:
        if not state_to_abbr.get(data_point[ind].upper(), False):
            continue
        state = state_to_abbr[data_point[ind].upper()]
        state_stats[state] = {}
        for k,v in stats_ind.items():
            state_stats[state][stats_dict[k]] = data_point[v]

url = url = "https://api.census.gov/data/2017/acs/acs5/profile?get="
url += ",".join(stats_dict.keys())
url += ",NAME&for=us:*"
url += '&key=' + census_key
response = requests.get(url)
if response.status_code < 400:
    data = response.json()
    ind = data[0].index("NAME")
    stats_ind = {s:data[0].index(s) for s in stats_dict.keys()}
    state_stats['USA'] = {}
    for k,v in stats_ind.items():
        state_stats['USA'][stats_dict[k]] = data_point[v]

200


In [11]:
stats_list = list(stats_dict.values())
def get_user_data(tweet):
    if (not tweet) or (not tweet.get('user', None)) or (not tweet['user']['location']):
        return None
    loc = match_string(tweet['user']['location'])
    if loc:
        if type(loc)==dict:
            out = [
                tweet['user']['id_str'],
                tweet['user']['friends_count'],
                tweet['user']['followers_count'],
                loc['state'],
                loc['name'],
                loc['latitude'],
                loc['longitude'],
                tweet['user']['created_at'],
                loc['density']
            ]
            for s in stats_list:
                out.append(loc.get(s, None))
        else:
            out = [
                tweet['user']['id_str'],
                tweet['user']['friends_count'],
                tweet['user']['followers_count'],
                loc,
                None,
                None,
                None,
                tweet['user']['created_at'],
                None
            ]
            for s in stats_list:
                out.append(state_stats.get(loc, {}).get(s, None))
        return out
    else:
        return None
    

In [12]:
from twitterArchiveParser import twitterArchiveParser

In [13]:
tp = twitterArchiveParser(archive_location=arc_name, save_location = save_name, num_threads = 7)

In [14]:
header_string = 'id,friends,followers,state,city,latitude,longitude,created_at,density,' + ','.join(stats_list)
tp.write_header(header_string)
tp.parse_archive(get_user_data)

Thread finished

Thread finished

Thread finished

Thread finished

Thread finished

Thread finished

Thread finished

